In [3]:
import os, time
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt

class preprocess:
    def __init__(self, file):
        self.name = file.split("_")[4]
        self.img0 = cv2.imread(r"C:\Users\James\Desktop\phone_data\data\\"+file) # ~1
        self.img = cv2.imread(r"C:\Users\James\Desktop\phone_data\data\\"+file, cv2.IMREAD_GRAYSCALE)
        self.img = cv2.resize(self.img, (self.img.shape[1]//10,self.img.shape[0]//10))
        #self.plot([self.img], ["decolorized"])
        self.canny()      # get contour image self.edged
        self.contour()    # get contour matrix self.maxContour with max size area
        self.boundRect()  # get boundary, rotation center and angle
        self.capture0()   # capture img0 -> total 3.8 sec # capture img -> total 1.5 sec
        self.cut()
    
    def plot(self, imgList, titleList=None, cmap="gray"):
        fig = plt.figure() if len(imgList)==1 else plt.figure(figsize=(20,5))
        fig.set_facecolor("white")
        for i in range(len(imgList)):
            plt.subplot(1,len(imgList),i+1)
            if titleList:
                plt.title(titleList[i])
            plt.imshow(imgList[i], cmap=cmap)
        plt.show()
    
    def canny(self):
        blurred = cv2.GaussianBlur(self.img.copy(), (11, 11), 0)
        self.edged = cv2.Canny(blurred, 30, 150)
        #self.plot([blurred, self.edged], ["blurred", "canny"])
    
    def contour(self):
        contours, hierarchy = cv2.findContours(self.edged.copy(), cv2.RETR_EXTERNAL, \
            cv2.CHAIN_APPROX_SIMPLE)
        contoursSize=[ (c[:,0,0].max()-c[:,0,0].min())*(c[:,0,1].max()-c[:,0,1].min()) \
            for c in contours ]
        self.maxContour = contours[ np.argmax(contoursSize) ]
        #cImg = np.zeros( (self.img.shape[0],self.img.shape[1],3) ).astype(np.uint8)
        #cImg[:,:,0], cImg[:,:,1], cImg[:,:,2] = self.img.copy(), self.img.copy(), self.img.copy()       
        #maxContourImg = cv2.drawContours(cImg, [self.maxContour], 0, (0,255,0), 3)
        #self.plot([maxContourImg], ["Contour"], cmap=None)
        
    def boundRect(self):
        self.minx, self.maxx = self.maxContour[:,0,0].min(), self.maxContour[:,0,0].max()
        self.miny, self.maxy = self.maxContour[:,0,1].min(), self.maxContour[:,0,1].max()
        recImg = self.img.copy()[self.miny:self.maxy,self.minx:self.maxx]
        (self.centerx, self.centery), (w, h), self.angle = cv2.minAreaRect(self.maxContour)
        self.angle = self.angle if self.angle<45 else 90-self.angle
        #rot_mat = cv2.getRotationMatrix2D((self.centerx,self.centery), self.angle, 1.0)
        #rotImg = cv2.warpAffine(recImg, rot_mat, recImg.shape[::-1], flags=cv2.INTER_LINEAR)
        #self.plot([recImg, rotImg], ["inscribed rect", "min inscribed rect after rotation"])
    
    def capture(self):
        recImg = self.img0[self.miny*10+500:self.maxy*10-500,self.minx*10+500:self.maxx*10-500,:]
        rot_mat = cv2.getRotationMatrix2D((self.centerx*10,self.centery*10), self.angle, 1.0)
        self.rotImg = cv2.warpAffine(recImg, rot_mat, (recImg.shape[1],recImg.shape[0]), flags=cv2.INTER_LINEAR)
        self.plot([self.rotImg], [self.name], cmap=None)
        #self.plot([self.img0, rotImg], ["inscribed rect", "min inscribed rect after rotation"])
    
    def capture0(self):
        rot_mat = cv2.getRotationMatrix2D((self.centerx*10,self.centery*10), self.angle, 1.0)
        shape = (self.maxy*10-self.miny*10-1000, self.maxx*10-self.minx*10-1000)
        recImgR = self.img0[self.miny*10+500:self.maxy*10-500,self.minx*10+500:self.maxx*10-500,0].reshape(shape) 
        recImgG = self.img0[self.miny*10+500:self.maxy*10-500,self.minx*10+500:self.maxx*10-500,1].reshape(shape)
        recImgB = self.img0[self.miny*10+500:self.maxy*10-500,self.minx*10+500:self.maxx*10-500,2].reshape(shape)
        
        rotImgR = cv2.warpAffine(recImgR, rot_mat, shape[::-1], flags=cv2.INTER_LINEAR)
        rotImgG = cv2.warpAffine(recImgG, rot_mat, shape[::-1], flags=cv2.INTER_LINEAR)
        rotImgB = cv2.warpAffine(recImgB, rot_mat, shape[::-1], flags=cv2.INTER_LINEAR)
        self.rotImg = np.zeros(shape+(3,)).astype(np.uint8)
        self.rotImg[:,:,0], self.rotImg[:,:,1], self.rotImg[:,:,2] = rotImgR, rotImgG, rotImgB # no reverse
        #self.plot([self.rotImg], [self.name], cmap=None)
        #self.plot([self.img0, rotImg], ["inscribed rect", "min inscribed rect after rotation"])
    
    def cut(self):
        h, w, _ = self.rotImg.shape
        leftImg  = [ self.rotImg[i*320:i*320+640,0:640,:] for i in range(h//320-1) ]
        rightImg = [ self.rotImg[i*320:i*320+640,w-640:w,:] for i in range(h//320-1) ] 
        upImg    = [ self.rotImg[0:640,i*320:i*320+640,:] for i in range(w//320-1) ]
        downImg  = [ self.rotImg[h-640:h,i*320:i*320+640,:] for i in range(w//320-1) ]
        totalImg = leftImg+rightImg+upImg+downImg+[self.rotImg[h-640:h,w-640:w,:]]
        #os.mkdir(f"cutImg/{self.name}")
        #for i in range(len(totalImg)):
        #    cv2.imwrite(f"cutImg/{self.name}/{self.name}_{i}.jpg", totalImg[i])
        #rows = int(len(totalImg)//4+((len(totalImg)%4)>0))
        #fig = plt.figure(figsize=(20,4*rows))
        #fig.set_facecolor("white")
        #for i,img in enumerate(totalImg):
        #    plt.subplot(rows, 4, i+1)
        #    plt.imshow(totalImg[i], cmap=None)
        #plt.show()
        
#f="11W86000383015312LKF070_1619408007_11W86000383015312LKF070_SA-AOI-TRIM_2021-04-26-11-33-27_org.jpg"
#g="11W86000383015312LKF0A0_1618553542_11W86000383015312LKF0A0_SA-AOI-TRIM_2021-04-16-14-12-22_org.jpg"

In [4]:
for i,file in enumerate(os.listdir(r"C:\Users\James\Desktop\phone_data\data\\")):
    s = time.time()
    obj = preprocess(file)
    print(i, round(time.time()-s,2), end=" || ")

0 1.64 || 

In [5]:
s = time.time()
cv2.imread(r"C:\Users\James\Desktop\phone_data\data\\"+file)
print(time.time()-s)

0.7984106540679932


In [ ]:
L = os.listdir(r"C:\Users\James\Desktop\phone_data\data\\")
print(len(L), L[0])
M = list(map(lambda s:s.split("_")[4], L))
print(len(M), M[0])